In [ ]:
cd /content/drive/MyDrive/study/EffectivePython/chp.2

/content/drive/MyDrive/study/EffectivePython/chp.2


# 2장 프로파일링으로 병목 지점 찾기

- 코드에서 RAM 병목 지점을 찾는 방법
- CPU와 메모리 사용량을 프로파일링하는 방법
- 바람직한 프로파일링의 깊이
- 장시간 실행되는 앱 프로파일링 방법
- C파이썬의 내부동작
- 성능 튜닝 중 코드의 올바름을 유지하는 방법

---
### 효과적으로 프로파일링하기

RAM을 많이 사용하는 부분을 찾는다.
테스트할 부분만 떼어내서 테스트한다.

- %timeit 또는 time.time()
- cProfile: 어떤 함수가 가장 오래 걸리는가
- line_profiler: 한 줄 씩 탐색 (장시간 프로세스: py-spy)
- memory_profiler: RAM 사용량이 높은 이유를 찾는다.
- 파이썬 바이트코드
- 프로파일링과 단위테스트

---

time.time()



In [ ]:
"""Julia set generator with timing decorator"""
import time
from functools import wraps

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193


def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    return measure_time


@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output


def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for 1000^2 grid with 300 iterations


# Calculate the Julia set using a pure Python solution with
# reasonable defaults for a laptop
# set draw_output to True to use PIL to draw an image
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 8.82549524307251 seconds
calculate_z_serial_purepython took 8.82620096206665 seconds


In [ ]:
# from IPython.display import Image
# Image(filename='test.png') 

---

[%timeit](https://docs.python.org/ko/3/library/timeit.html)

- 일시적으로 GC를 비활성화 한다.

In [ ]:
!python -m timeit -n 5 -r 1 -s "import julia1" "julia1.calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)"

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.671994686126709 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.461464881896973 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.454515218734741 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.5059814453125 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.4686291217803955 seconds
5 loops, best of 1: 8.09 sec per loop


---

cProfile


In [ ]:
!python -m cProfile -s cumulative julia1_nopil.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 11.773769617080688 seconds
         36221995 function calls in 12.635 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   12.635   12.635 {built-in method builtins.exec}
        1    0.043    0.043   12.635   12.635 julia1_nopil.py:1(<module>)
        1    0.628    0.628   12.592   12.592 julia1_nopil.py:23(calc_pure_python)
        1    8.510    8.510   11.774   11.774 julia1_nopil.py:9(calculate_z_serial_purepython)
 34219980    3.264    0.000    3.264    0.000 {built-in method builtins.abs}
  2002000    0.184    0.000    0.184    0.000 {method 'append' of 'list' objects}
        1    0.006    0.006    0.006    0.006 {built-in method builtins.sum}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 {built-in method time.time}
        4    0.000   

In [ ]:
import pstats
p = pstats.Stats("profile.stats")
p.sort_stats("cumulative")

In [ ]:
p.print_stats()

Thu Jul 15 04:52:13 2021    profile.stats

         36221995 function calls in 15.299 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.299   15.299 {built-in method builtins.exec}
        1    0.047    0.047   15.299   15.299 julia1_nopil.py:1(<module>)
        1    0.497    0.497   15.252   15.252 julia1_nopil.py:23(calc_pure_python)
        1   10.131   10.131   14.552   14.552 julia1_nopil.py:9(calculate_z_serial_purepython)
 34219980    4.421    0.000    4.421    0.000 {built-in method builtins.abs}
  2002000    0.195    0.000    0.195    0.000 {method 'append' of 'list' objects}
        1    0.007    0.007    0.007    0.007 {built-in method builtins.sum}
        3    0.001    0.000    0.001    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        2    0.000    0.000    0.000    0.000 {built-in method time.time}
     

In [ ]:
p.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
{built-in method builtins.exec}                   <- 
julia1_nopil.py:1(<module>)                       <-       1    0.047   15.299  {built-in method builtins.exec}
julia1_nopil.py:23(calc_pure_python)              <-       1    0.497   15.252  julia1_nopil.py:1(<module>)
julia1_nopil.py:9(calculate_z_serial_purepython)  <-       1   10.131   14.552  julia1_nopil.py:23(calc_pure_python)
{built-in method builtins.abs}                    <- 34219980    4.421    4.421  julia1_nopil.py:9(calculate_z_serial_purepython)
{method 'append' of 'list' objects}               <- 2002000    0.195    0.195  julia1_nopil.py:23(calc_pure_python)
{built-in method builtins.sum}                    <-       1    0.007    0.007  julia1_nopil.py:23(calc_pure_python)
{built-in method builtins.print}                  <-       3    0.0

In [ ]:
p.print_callees()

   Ordered by: cumulative time

Function                                          called...
                                                      ncalls  tottime  cumtime
{built-in method builtins.exec}                   ->       1    0.047   15.299  julia1_nopil.py:1(<module>)
julia1_nopil.py:1(<module>)                       ->       1    0.497   15.252  julia1_nopil.py:23(calc_pure_python)
julia1_nopil.py:23(calc_pure_python)              ->       1   10.131   14.552  julia1_nopil.py:9(calculate_z_serial_purepython)
                                                           2    0.000    0.000  {built-in method builtins.len}
                                                           3    0.001    0.001  {built-in method builtins.print}
                                                           1    0.007    0.007  {built-in method builtins.sum}
                                                           2    0.000    0.000  {built-in method time.time}
                                

---

 SnakeViz로 cProfile 결과 시각화
 

In [ ]:
!python -m snakeviz -s profile.stats

/usr/bin/python3: No module named snakeviz


---

 line_profiler
 

In [ ]:
!pip install line-profiler

     |████████████████████████████████| 63 kB 2.3 MB/s 


In [ ]:
!kernprof -l -v julia1_lineprofiler.py

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 93.70192384719849 seconds
Wrote profile results to julia1_lineprofiler.py.lprof
Timer unit: 1e-06 s

Total time: 53.0277 s
File: julia1_lineprofiler.py
Function: calculate_z_serial_purepython at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           @profile
    10                                           def calculate_z_serial_purepython(maxiter, zs, cs):
    11                                               """Calculate output list using Julia update rule"""
    12         1       2246.0   2246.0      0.0      output = [0] * len(zs)
    13   1000001     423738.0      0.4      0.8      for i in range(len(zs)):
    14   1000000     397221.0      0.4      0.7          n = 0
    15   1000000     451012.0      0.5      0.9          z = zs[i]   
    16   1000000     434059.0      0.4      0.8          c = cs[i]
    17  34219980   20192870.0      0.6 

---

memory_profiler


In [ ]:
def lis():
  [i*i for i in range(100000)]

In [ ]:
def tup():
  (i*i for i in range(100000))

In [ ]:
!python -m memory_profiler lis()

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `python -m memory_profiler lis()'


---

PySPy



In [ ]:
!ps -A -o pid,rss,cmd | ack python

/bin/bash: ack: command not found


---

바이트코드: 내부 작동



In [ ]:
import dis
import julia1_nopil
dis.dis(julia1_nopil.calculate_z_serial_purepython)

 11           0 LOAD_CONST               1 (0)
              2 BUILD_LIST               1
              4 LOAD_GLOBAL              0 (len)
              6 LOAD_FAST                1 (zs)
              8 CALL_FUNCTION            1
             10 BINARY_MULTIPLY
             12 STORE_FAST               3 (output)

 12          14 SETUP_LOOP              94 (to 110)
             16 LOAD_GLOBAL              1 (range)
             18 LOAD_GLOBAL              0 (len)
             20 LOAD_FAST                1 (zs)
             22 CALL_FUNCTION            1
             24 CALL_FUNCTION            1
             26 GET_ITER
        >>   28 FOR_ITER                78 (to 108)
             30 STORE_FAST               4 (i)

 13          32 LOAD_CONST               1 (0)
             34 STORE_FAST               5 (n)

 14          36 LOAD_FAST                1 (zs)
             38 LOAD_FAST                4 (i)
             40 BINARY_SUBSCR
             42 STORE_FAST               6 (z)

 15   

In [ ]:
def fn_expressive(upper=1_000_000):
    total = 0
    for n in range(upper):
        total += n
    return total

def fn_terse(upper=1_000_000):
    return sum(range(upper))

assert fn_expressive() == fn_terse(), "Expect identical results from both functions"

In [ ]:
%timeit fn_expressive()

10 loops, best of 5: 72.9 ms per loop


In [ ]:
%timeit fn_terse()

10 loops, best of 5: 21.8 ms per loop


In [ ]:
import dis
dis.dis(fn_expressive)

  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (total)

  3           4 SETUP_LOOP              24 (to 30)
              6 LOAD_GLOBAL              0 (range)
              8 LOAD_FAST                0 (upper)
             10 CALL_FUNCTION            1
             12 GET_ITER
        >>   14 FOR_ITER                12 (to 28)
             16 STORE_FAST               2 (n)

  4          18 LOAD_FAST                1 (total)
             20 LOAD_FAST                2 (n)
             22 INPLACE_ADD
             24 STORE_FAST               1 (total)
             26 JUMP_ABSOLUTE           14
        >>   28 POP_BLOCK

  5     >>   30 LOAD_FAST                1 (total)
             32 RETURN_VALUE


---

최적화 중에 단위 테스트



In [ ]:
import time

def test_some_fn():
    """Check basic behaviours for our function"""
    assert some_fn(2) == 4
    assert some_fn(1) == 1
    assert some_fn(-1) == 1


# check for line_profiler or memory_profiler in the local scope, both
# are injected by their respective tools or they're absent
# if these tools aren't being used (in which case we need to substite
# a dummy @profile decorator)
if 'line_profiler' not in dir() and 'profile' not in dir():
    def profile(func):
        def inner(*args, **kwargs):
            return func(*args, **kwargs)
        return inner

@profile
def some_fn(useful_input):
    """An expensive function that we wish to both test and profile"""
    # artificial 'we're doing something clever and expensive' delay
    time.sleep(1) 
    return useful_input ** 2

print(f"Example call `some_fn(2)` == {some_fn(2)}")

Example call `some_fn(2)` == 4


In [ ]:
!pytest utility.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content/drive/My Drive/study/EffectivePython/chp.2, inifile:
collected 0 items / 1 errors                                                   

==================================== ERRORS ====================================
_________________________ ERROR collecting utility.py __________________________
/usr/local/lib/python2.7/dist-packages/_pytest/python.py:468: in _importtestmodule
    mod = self.fspath.pyimport(ensuresyspath=importmode)
/usr/local/lib/python2.7/dist-packages/py/_path/local.py:701: in pyimport
    __import__(modname)
E     File "/content/drive/My Drive/study/EffectivePython/chp.2/utility.py", line 29
E       print(f"Example call `some_fn(2)` == {some_fn(2)}")
E                                                        ^
E   SyntaxError: invalid syntax
!!!!!!!!!!!!!!!!!!! Interrupted: 1 errors during collection 